# CETM50 -  Technology Management For Organisations
## Name: AKINWUMI ISAAC KAYODE

## Student ID: bi54up


In [1]:
import csv
import json
import xml.etree.ElementTree as ET

## Function to parse CSV data, merge JSON data, merge XML data, read text file

In [2]:
def parse_csv_data(file_path):
    """
    Parses CSV data and returns a dictionary.

    :param file_path: Path to the CSV file.
    :return: Dictionary containing parsed CSV data.
    """
    parsed_data = {}
    with open(file_path, newline='') as csvfile:
        csv_reader = csv.DictReader(csvfile)
        for row in csv_reader:
            full_name = row['First Name'] + ' ' + row['Second Name']
            parsed_data[full_name] = row
    return parsed_data

In [3]:
def merge_json_data(file_path, csv_data):
    """
    Merges JSON data with existing CSV data.

    :param file_path: Path to the JSON file.
    :param csv_data: Existing CSV data dictionary.
    """
    with open(file_path) as jsonfile:
        json_data = json.load(jsonfile)
        for record in json_data:
            full_name = record['firstName'] + ' ' + record['lastName']
            # Handle 'debt' field
            if 'debt' in record and isinstance(record['debt'], dict):
                debt_info = record['debt']
                dept_amount = debt_info.get('amount', 0)
                dept_time_period_years = debt_info.get('time_period_years', 0)
                del record['debt']
            else:
                dept_amount = 0
                dept_time_period_years = 0
            record['dept_amount'] = dept_amount
            record['dept_time_period_years'] = dept_time_period_years
            csv_data.setdefault(full_name, {}).update(record)

In [4]:
def merge_xml_data(file_path, csv_data):
    """
    Merges XML data with existing CSV data.

    :param file_path: Path to the XML file.
    :param csv_data: Existing CSV data dictionary.
    """
    xml_data = ET.parse(file_path).getroot()
    for user in xml_data.findall('user'):
        full_name = f"{user.attrib['firstName']} {user.attrib['lastName']}"
        record = {attr: user.attrib[attr] for attr in user.attrib}
        csv_data.setdefault(full_name, {}).update(record)

In [5]:
def read_text_file(file_path):
    """
    Reads a text file line by line and returns the data in a list.

    :param file_path: Path to the text file.
    :return: List containing each line of the text file.
    """
    data = []
    with open(file_path, mode='r', encoding='utf-8') as file:
        data = file.readlines()
    return data

In [17]:
# Read text file
txt_data = read_text_file('cetm50_23_4_data/user_data_23_4.txt')
for i in range(0, 4):
    print(txt_data[i])

# Parse CSV data
csv_data = parse_csv_data('cetm50_23_4_data/user_data_23_4.csv')

# Merge JSON data
merge_json_data('cetm50_23_4_data/user_data_23_4.json', csv_data)

# Merge XML data
merge_xml_data('cetm50_23_4_data/user_data_23_4.xml', csv_data)

"Valerie Ellis e-mailed in overnight (Full details in Ticket #1839). During account creation something went wrong and their security code is wrong on their billing information. Bank is rejecting any payment until it's corrected. They're not sure what happened, but said to try "762". Can you please action this and try re-bill the client? Let me know if there's any further issues with it. Thanks"

"Congratulations on the promotion West! We wouldn't have survived through the pandemic without you and your team. As a token of our appreciation, we've given you a £2100 salary bump. This will take effect as of next month's payroll. At Williams-Wheeler we value the care and work you put in. See you on Monday Charlie, enjoy!"

"Happy Birthday Mr Charlie Short! You're 52 today. Our latest offers will be sure to get you into the party spirit!"

"Hannah, the pension policy has changed slightly since the meeting last week. I know you've just finished putting through all the changes, but I need you t

In [18]:

# Convert merged dictionary into a unified dataset format
unified_data = list(csv_data.values())

# Define updates based on conditions
updates = {
    ('Valerie', 'Ellis'): {'credit_card_security_code': 762},
    ('Charlie', 'West'): {'salary': 2100, 'age': 52},
    ('Martin', None): {'pension': lambda x: 0.0015 * int(x) + int(x)}  # lambda function for dynamic update
}

# Apply updates
for record in unified_data:
    for (first_name, last_name), update_fields in updates.items():
        if record.get('firstName') == first_name and record.get('lastName') == last_name:
            for field, value in update_fields.items():
                if callable(value):
                    record[field] = value(record.get(field, 0))
                else:
                    record[field] = value
            print(f"{', '.join(update_fields.keys())} Updated")

# Drop specified columns from each record
columns_to_drop = ['First Name', 'Second Name', 'Age (Years)', 'Sex', 'Vehicle Make', 'Vehicle Model', 'Vehicle Year', 'Vehicle Type', 'debt']
for record in unified_data:
    record['Vehicle_Make'] = record['Vehicle Make']
    record['Vehicle_Model'] = record['Vehicle Model']
    record['Vehicle_Year'] = record['Vehicle Year']
    record['Vehicle_Type'] = record['Vehicle Type']
    for column in columns_to_drop:
        record.pop(column, None)

# Write unified data to JSON
with open('customer_unified_data.json', 'w') as jsonfile:
    json.dump(unified_data, jsonfile, indent=4)


salary, age Updated
credit_card_security_code Updated


In [23]:
#unified_data

In [8]:
# ! pip install pony pymysql

In [21]:
from pony.orm import *

# Database configuration
host = 'europa.ashley.work'
user_and_database = 'student_bi54up'
password = 'iE93F2@8EhM@1zhD&u9M@K'

# Set up Pony ORM database
db = Database()

# Bind the database with MySQL
db.bind(provider='mysql', host=host,
        user=user_and_database,
        passwd=password,
        database=user_and_database)

# Enable SQL debug mode
sql_debug(True)

# db.drop_table("customerentity", with_all_data=True, if_exists=True)
########################################################################
class CustomerEntity(db.Entity):
    """
    Pony ORM model of records table
    """
    id = PrimaryKey(int, auto=True)
    firstName = Required(str)
    lastName = Required(str)
    age = Required(int)
    sex = Required(str)
    address_main = Required(str)
    address_city = Required(str)
    Vehicle_Make = Required(str)
    Vehicle_Model = Required(str)
    Vehicle_Year = Required(int)
    Vehicle_Type = Required(str)
    retired = Optional(bool)
    credit_card_number = Required(str)
    credit_card_security_code = Required(int)
    credit_card_start_date = Required(str)
    credit_card_end_date = Required(str)
    dept_amount = Optional(float)
    dept_time_period_years = Optional(float)
    dependants = Optional(str)
    marital_status = Required(str)
    salary = Required(int)
    pension = Optional(float)
    company = Optional(str)
    commute_distance = Optional(float)
    address_postcode = Optional(str)
    iban = Optional(str)

db.generate_mapping(create_tables=True) 
sql_debug(True)


def push_data(db_class, data):
  """
    Uploads data to the specified database table.

    :param db_class: Pony ORM entity class representing the database table.
    :param data: List of dictionaries representing the data to be uploaded.
  """
  try:
    for entry in data:
        db_class(**entry)
    commit()
    print('Entries Added Successfully')
  except Exception as e:
        print(f"An error occurred while uploading data to the database: {e}")


GET NEW CONNECTION
RELEASE CONNECTION
GET CONNECTION FROM THE LOCAL POOL
SET foreign_key_checks = 0
CREATE TABLE `customerentity` (
  `id` INTEGER PRIMARY KEY AUTO_INCREMENT,
  `firstname` VARCHAR(255) NOT NULL,
  `lastname` VARCHAR(255) NOT NULL,
  `age` INTEGER NOT NULL,
  `sex` VARCHAR(255) NOT NULL,
  `address_main` VARCHAR(255) NOT NULL,
  `address_city` VARCHAR(255) NOT NULL,
  `vehicle_make` VARCHAR(255) NOT NULL,
  `vehicle_model` VARCHAR(255) NOT NULL,
  `vehicle_year` INTEGER NOT NULL,
  `vehicle_type` VARCHAR(255) NOT NULL,
  `retired` BOOLEAN,
  `credit_card_number` VARCHAR(255) NOT NULL,
  `credit_card_security_code` INTEGER NOT NULL,
  `credit_card_start_date` VARCHAR(255) NOT NULL,
  `credit_card_end_date` VARCHAR(255) NOT NULL,
  `dept_amount` DOUBLE,
  `dept_time_period_years` DOUBLE,
  `dependants` VARCHAR(255) NOT NULL,
  `marital_status` VARCHAR(255) NOT NULL,
  `salary` INTEGER NOT NULL,
  `pension` DOUBLE,
  `company` VARCHAR(255) NOT NULL,
  `commute_distance` DO

In [22]:
push_data(CustomerEntity, unified_data)

GET NEW CONNECTION
INSERT INTO `customerentity` (`firstname`, `lastname`, `age`, `sex`, `address_main`, `address_city`, `vehicle_make`, `vehicle_model`, `vehicle_year`, `vehicle_type`, `retired`, `credit_card_number`, `credit_card_security_code`, `credit_card_start_date`, `credit_card_end_date`, `dept_amount`, `dept_time_period_years`, `dependants`, `marital_status`, `salary`, `pension`, `company`, `commute_distance`, `address_postcode`, `iban`) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
['Leon', 'Smith', 48, 'Male', '0 Shaw plains', "O'Neillshire", 'Mercedes-Benz', 'Astro Cargo', 2009, 'Pickup', True, '2292406027303835', 750, '09/18', '01/22', 0.0, 0.0, '2', 'married or civil partner', 65911, 0.0, 'Gilbert, Williams and Quinn', 15.68, 'DY0 3SG', 'GB05DRJP98363742599825']

INSERT INTO `customerentity` (`firstname`, `lastname`, `age`, `sex`, `address_main`, `address_city`, `vehicle_make`, `vehicle_model`, `vehicle_year`, `